In [0]:
import pandas as pd
import urllib.request
import psycopg2
import pandas as pd



In [0]:
# To get the list of subject_ids for whom waveform exists in mimic matched waveform database.

column = ['subject_id']
df1 = pd.DataFrame(columns= column);    
i=0;
data =  urllib.request.urlopen("https://archive.physionet.org/physiobank/database/mimic3wdb/matched/RECORDS");    
for lines in data.readlines():
  data = lines.decode("utf-8");
  #print (data) ;
  #print ( data[5:11]);
  df1.loc[i] = data[5:11]
  i =i +1;

print (df1);

In [0]:
# To connect to the mimic DB

conn = psycopg2.connect(database="mimic",user="postgres",password="postgres",host="localhost", port="5432")
cur = conn.cursor()
cur.execute("SET search_path TO " + "mimiciii")

In [0]:
#Query to fetch subject IDs from the selected cohort in DB.
query_subjectids_coh = """select icu.subject_id, icu.icustay_id   from 
sepsis3_cohort coh, icustays icu
where coh.icustay_id= icu.icustay_id and coh.excluded=0 """;

df2 = pd.read_sql(query_subjectids_coh, conn)
print(df2);

In [0]:
df1 = df1.astype(int)
df2 = df2.astype(int)
# Inner join both dataframes on subject_id
df3 = pd.merge(df1, df2, on='subject_id', how='inner');
print (df3.shape[0])
print (df3);

In [0]:
# For all the subject_ids for which the waveform exists in mimic matched waveform database, set the value of column 'waveform_exists' to '1'
for index, row in df3.iterrows():
  Query_update_sepsis3coh = ("update sepsis3_cohort coh set waveform_exists = 1 where coh.icustay_id =" + str(row['icustay_id']));
  print(Query_update_sepsis3coh);
  cur.execute(Query_update_sepsis3coh);

In [0]:
# to commit the updated records
conn.commit();

In [146]:
# To check how many patients with sepsis (suspected infection +change in SOFA >=2) have waveform records in the mimic matched waveform db

conn = psycopg2.connect(database="mimic",user="postgres",password="postgres",host="localhost", port="5432")
cur = conn.cursor()
cur.execute("SET search_path TO " + "mimiciii")

query_count_Sepsis_waveform_exists = "select count(*) from sepsis3_cohort coh where coh.waveform_exists = 1 and coh.has_sepsis=1";
cur.execute(query_count_Sepsis_waveform_exists);
result = cur.fetchall();
for i in result:
  print (i);

# To generate a CSV file consisting for all the subject_ids from selected cohort for which the waveform exists in the mimic matched waveform db
query_waveform_exists = "select subject_id,icustay_id,sepsis_onsettime from sepsis3_cohort coh where coh.waveform_exists = 1";
df_waveform_exists = pd.read_sql(query_waveform_exists,conn);
print(df_waveform_exists.shape[0]);


df_waveform_exists.to_csv ('df_waveform_exists.csv', sep=',', index = False, header=True);


(2325,)
4653
